In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *
import tensorflow_addons as tfa
from sklearn.metrics import classification_report

In [2]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
test = pd.read_csv("../../dataset/goodreads_test.csv")
vocabulary2 = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)

In [3]:
# lenght = 0
# list_word = []
# list_weight = []
# with open('../dataset/glove.6B.300d.txt', encoding='utf-8') as f:
#     for line in tqdm(f):
#         values = line.split()
#         word = values[0]#.replace('<','').replace('>', '')
#         coefs = values[1:]
#         if len(coefs) == 300:
#             #list_weight.append(coefs)
#             list_word.append(word)
#             lenght += 1
#         else:
#             print("incorect dim")

In [4]:
#model_list = [cnn1,cnn2, cnn3,cnn4,cnn5,cnn6, cnn8, cnn9, cnn10]
model_list = [unet1]
def scheduler(epoch, lr):
    if epoch < 4:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
for seed in [8,5]:
    keras.utils.set_random_seed(seed)
    for model_obj in model_list:
        model = model_obj.Model(vocabulary2)
        model.model.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                           loss=keras.losses.categorical_crossentropy,
                           metrics=[keras.metrics.categorical_accuracy, tfa.metrics.F1Score(num_classes=6, average='weighted')]
                           )
        print(model.name)
        #print(model.model.summary())
        if not os.path.exists(f"logs/{model.name}"):
            os.mkdir(f"logs/{model.name}")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{model.name}")
        model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=9, callbacks=[keras.callbacks.LearningRateScheduler(scheduler,0)], batch_size=60, validation_split=0.2)

unet1
Epoch 1/9
12000/12000 [==============================] - 212s 17ms/step - loss: 1.1997 - categorical_accuracy: 0.4869 - f1_score: 0.4671 - val_loss: 1.0371 - val_categorical_accuracy: 0.5542 - val_f1_score: 0.5336 - lr: 0.0010
Epoch 2/9
12000/12000 [==============================] - 207s 17ms/step - loss: 1.0472 - categorical_accuracy: 0.5552 - f1_score: 0.5444 - val_loss: 1.0000 - val_categorical_accuracy: 0.5718 - val_f1_score: 0.5622 - lr: 0.0010
Epoch 3/9
12000/12000 [==============================] - 205s 17ms/step - loss: 1.0072 - categorical_accuracy: 0.5737 - f1_score: 0.5660 - val_loss: 0.9846 - val_categorical_accuracy: 0.5775 - val_f1_score: 0.5686 - lr: 0.0010
Epoch 4/9
12000/12000 [==============================] - 208s 17ms/step - loss: 0.9804 - categorical_accuracy: 0.5863 - f1_score: 0.5801 - val_loss: 0.9737 - val_categorical_accuracy: 0.5835 - val_f1_score: 0.5768 - lr: 0.0010
Epoch 5/9
12000/12000 [==============================] - 210s 18ms/step - loss: 0.9548

In [5]:
res = model.model.predict([train['review_text'], train['n_comments'], train['n_votes']])

28125/28125 [==============================] - 164s 6ms/step


In [ ]:
restest = model.model.predict([test['review_text'], test['n_comments'], test['n_votes']])

In [ ]:
ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
test_data = np.array(ff)

ff = []
for line in tqdm(res):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
train_data = np.array(ff)

In [ ]:

print(classification_report(train['rating'], train_data))

In [ ]:
test['rating'] = test_data

id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])
df['review_id'] = id
df['rating'] = rating

In [ ]:
df.to_csv('submission_unet1_embedding_class_weights_model.csv',index=False )
